# Mini-GPT (referencing from Andrej Karpathy)
[Related link](https://www.youtube.com/watch?v=kCc8FmEb1nY&list=LL&index=5&t=5627s)

get dataset via `wget`

In [6]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
print(len(text))

1115393


In [8]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## Preprocessing

In [10]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [12]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda I: ''.join([itos[i] for i in I])

print(encode("Hello World!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [13]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([1115393])


C:\Users\user\AppData\Local\Temp\ipykernel_21904\3699282215.py:2: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  data = torch.tensor(encode(text), dtype=torch.long)


## Split train and validation set

In [14]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

## divide data into feature and label in order to make it trainable

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
x, y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [22]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
xb, yb = get_batch("train")
print(xb.shape)

torch.Size([4, 8])


`nn.Embedding(num_embeddings, embedding_dim)`
* num_embeddings: embedding이 이루어지는 원소의 개수
* embedding_dim: embedding된 각 원소의 크기

`torch.multinomial(probs, num_samples)`
* probs: distribution
* num_samples:최대값을 몇개 뽑을 것인지

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, target=None):
        logits = self.token_embedding_table(idx) #idx(B, T) -> (B, T, C)

        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss
    def generate(self, idx, max_new_tokens):

        for _ in range(max_new_tokens):

            logits, loss = self(idx)
            logits = logits[:, -1, :] # focus on the last one
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [31]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


## Let's trains the model

In [32]:
import torch.optim as optim

optimizer = optim.AdamW(m.parameters(), lr=1e-3)

In [36]:
batch_size = 32
epochs= 100
for steps in range(epochs):
    xb, yb = get_batch('train')

    logits, loss = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

4.5992255210876465


이를 통해서 Bigram에서 AdamW를 사용한 경사하강법은 대략 4.6의 loss를 가지게 된다
`zero_grad(set_to_none=True)`에서 `set_to_none`의 의미는?

## Mathematical Trick in self-attention

In [41]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
print(a)
a = a / torch.sum(a, 1, keepdim=True)
print(a)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
